In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.model_selection as model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import pickle

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm, tree
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from tqdm import tqdm #creates progress bar to let you know how long is left till function is complete
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTENC
from imblearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('../data/cases_train_processed.csv')
df = df.drop(['Unnamed: 0','index','source','additional_information','Last_Update','Lat_right','Long_right','Province_State','Country_Region','dist_between_in_km'],1)
df['Confirmed'].fillna(df['Confirmed'].mean(),inplace=True)
df['Deaths'].fillna(df.Deaths.mean(),inplace=True)
df['Recovered'].fillna(df.Recovered.mean(),inplace=True)
df['Active'].fillna(df.Active.mean(),inplace=True)
df['Incidence_Rate'].fillna(df.Incidence_Rate.mean(),inplace=True)
df['Case-Fatality_Ratio'].fillna(df['Case-Fatality_Ratio'].mean(),inplace=True)
df.date_confirmation = pd.to_datetime(df.date_confirmation,infer_datetime_format=True) 
df.Combined_Key.fillna((df.province+" ,"+df.country),inplace=True)

In [4]:
df.age = df.age.astype('object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367634 entries, 0 to 367633
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   age                  367634 non-null  object        
 1   sex                  367634 non-null  object        
 2   province             367634 non-null  object        
 3   country              367634 non-null  object        
 4   date_confirmation    367634 non-null  datetime64[ns]
 5   outcome              367634 non-null  object        
 6   Confirmed            367634 non-null  float64       
 7   Deaths               367634 non-null  float64       
 8   Recovered            367634 non-null  float64       
 9   Active               367634 non-null  float64       
 10  Combined_Key         367634 non-null  object        
 11  Incidence_Rate       367634 non-null  float64       
 12  Case-Fatality_Ratio  367634 non-null  float64       
dtypes: datetime64[

---------------------------------------------------------------------

# TEsting Over and Under Sampling 
----------------------------------------------------------

In [6]:
def dropDuplicates(data):
    duplicates = data.columns[data.columns.duplicated()]
    if len(duplicates) > 0:
        data = data.loc[:,~data.columns.duplicated()]
    return data

def colsToDrop(dataframe):
    if 'outcome' in dataframe.columns:
        dataframe = dataframe.drop('outcome',1)
        print("dropping the outcome column")
    if 'Combined_Key' in dataframe.columns:
        dataframe = dataframe.drop('Combined_Key',1)
        print("dropping Combined_Key")
    if 'dist_between_in_km' in dataframe.columns:
        dataframe = dataframe.drop('dist_between_in_km',1)
        print("dropping dist in km column")
    if 'date_confirmation' in dataframe.columns:
        dataframe = dataframe.drop('date_confirmation',1)
        print("dropping date")
    return dataframe
    # once hot encode and add new cols to dataframe
def oneHotEncode_df(dataframe):
    dataframe = colsToDrop(dataframe)
    col2Encode = list(dataframe.select_dtypes(include=['object'])) #gets a list of all the features that are objects assumption is that those are categorical
    dummies = pd.get_dummies(dataframe,columns=col2Encode,prefix=col2Encode,sparse=True)
    res = pd.concat([dataframe, dummies], axis=1)
    #if we decide to drop one hot encoded values
    res = res.drop(col2Encode, axis=1)
    output = dropDuplicates(res)
    return output

In [7]:
#sample before ohe
def getCategoricalIndices(dataframe):
    #get the indices of all the categorical variables
    numericalVar = list(dataframe.select_dtypes(include=['float64']))
    catlist = list(dataframe.columns.difference(numericalVar))
    print(catlist)
    indlist = []
    for i in catlist:
        indlist.append(dataframe.columns.get_loc(i))
    return indlist

In [8]:
sparse_df = colsToDrop(df)
catIndList = getCategoricalIndices(sparse_df)

dropping the outcome column
dropping Combined_Key
dropping date
['age', 'country', 'province', 'sex']


In [9]:
%%time
#oversample 
oversample = SMOTENC(categorical_features=catIndList,random_state=0,sampling_strategy='not majority')
x_o,y_o = oversample.fit_resample(sparse_df,df.outcome)

In [1]:
len(x_o)

NameError: name 'x_o' is not defined

In [17]:
%%time
# add dummy variables  
ohe_df = oneHotEncode_df(df)
ohe_df

dropping the outcome column
dropping Combined_Key
dropping date
CPU times: user 744 ms, sys: 10.9 ms, total: 755 ms
Wall time: 753 ms


,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,age_1,age_2,age_3,age_4,...,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Uruguay,country_Venezuela,country_Vietnam,country_Zambia,country_Zimbabwe
0,238828.000000,4907.000000,201671.000000,32250.000000,1276.409575,2.054617,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,342788.000000,4869.000000,270094.000000,67825.000000,144.099577,1.420411,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2859.000000,2.000000,2639.000000,218.000000,464.331421,0.069955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,46779.000000,1871.000000,41228.000000,3680.000000,422.592353,3.999658,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,120336.000000,3286.000000,100974.000000,16076.000000,188.400627,2.730687,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367629,106331.902334,2739.064287,77838.956438,25753.923921,968.698456,2.721038,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
367630,238828.000000,4907.000000,201671.000000,32250.000000,1276.409575,2.054617,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
367631,2859.000000,2.000000,2639.000000,218.000000,464.331421,0.069955,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
367632,57558.000000,27.000000,57142.000000,389.000000,983.839751,0.046909,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#get the indices of all the categorical variables
numericalVar = list(ohe_df.select_dtypes(include=['float64']))
catlist = list(ohe_df.columns.difference(numericalVar))
indlist = []
for i in catlist:
    indlist.append(ohe_df.columns.get_loc(i))

In [19]:
len(indlist)

1249

In [20]:
x = ohe_df.copy()
y = df.outcome.copy()

/home/olay/Cmpt459/DoR459_Proj/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [1]:
len(y_o)

NameError: name 'y_o' is not defined

In [35]:
%%time
#transform data
oversample = SMOTENC(categorical_features=indlist,random_state=0,sampling_strategy='not majority')
undersample = RandomUnderSampler()
rfclassifier = RandomForestClassifier(n_estimators=25,random_state=0)
#define pipeline
steps = [('over',oversample),('under',undersample),('model',rfclassifier)]
pipeline = Pipeline(steps=steps)
#eval pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2,random_state=0)
scores = cross_val_score(pipeline,x,y,scoring='f1',cv=cv,n_jobs=1)
print('Mean F1: %.3f' %mean(scores))


Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.

In [1]:
ohe_np = ohe_df.to_numpy()
ohe_np
# summ = 0
# for i in range(len(enc.categories_)):
#     print(len(enc.categories_[i]))
#     summ+=len(enc.categories_[i])
# print('Total Sum is ',summ)

NameError: name 'enc' is not defined

# END oF SAMPLING SECTION
#########################################################################################################################33

In [4]:
res = enc.fit(df.drop('outcome',1))
res = res.transform(df.drop('outcome',1)).toarray()

In [57]:
X = res
y = df['outcome'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
xgb_model = xgb.XGBClassifier(n_estimators=50)
xgb_model.fit(X_train,y_train)
pickle.dump( xgb_model, open( "../models/xgboost50.pkl", "wb" ) )

C:\Users\Oliver Fujiki\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:17:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [58]:
trueXGBoost = pickle.load( open( '../models/xgboost50.pkl', "rb" ) )
XGprediction = list(trueXGBoost.predict(X_test))
xgConfusionMatrix = confusion_matrix(y_test, XGprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [59]:
pd.DataFrame(xgConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,5888,376,11440,7
nonhospitalized,92,29942,10,5
hospitalized,999,5,23881,3
deceased,87,79,683,30


In [46]:
clf = AdaBoostClassifier(n_estimators=10, random_state=0)
clf.fit(X_train, y_train)
pickle.dump( clf, open( "../models/AdaBoost10.pkl", "wb" ) )

In [63]:
trueAdaBoost = pickle.load( open( '../models/AdaBoost10.pkl', "rb" ) )
adaBoostprediction = list(trueAdaBoost.predict(X_test))
adaConfusionMatrix = confusion_matrix(y_test, adaBoostprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [64]:
pd.DataFrame(adaConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,843,782,16086,0
nonhospitalized,3,30046,0,0
hospitalized,531,38,24319,0
deceased,23,140,716,0


In [60]:
clf = RandomForestClassifier(n_estimators=25, random_state=0) 
clf.fit(X_train, y_train)
pickle.dump( clf, open( "../models/RandomForest25.pkl", "wb" ) )

In [61]:
trueRandomForest = pickle.load( open( '../models/RandomForest25.pkl', "rb" ) )
randomForestprediction = list(trueRandomForest.predict(X_test))
randomForestConfusionMatrix = confusion_matrix(y_test, randomForestprediction, labels=["recovered","nonhospitalized","hospitalized","deceased"])

In [62]:
pd.DataFrame(randomForestConfusionMatrix).rename(index = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"}, columns = {0:"recovered", 1: "nonhospitalized", 2: "hospitalized", 3: "deceased"})

,recovered,nonhospitalized,hospitalized,deceased
recovered,10562,174,6955,20
nonhospitalized,182,29833,13,21
hospitalized,3324,4,21548,12
deceased,256,55,528,40
